# Simple Plotter

In [1]:
import os, multiprocessing, sys
import copy
import math
from array import array
from ROOT import ROOT, gROOT, gStyle, gRandom, TSystemDirectory
from ROOT import TFile, TChain, TTree, TCut, TH1, TH1F, TH2F, THStack, TGraph, TGraphAsymmErrors
from ROOT import TStyle, TCanvas, TPad
from ROOT import TLegend, TLatex, TText, TLine, TBox
import time

cwd=os.getcwd()
sys.path.append(cwd+"/Utils/")

from drawLambda import *
from variables import *
from selections_SSLep import *
from samplesVH import *

gROOT.SetBatch(True)
gStyle.SetOptStat(0)
gROOT.Macro('functions.C')

Welcome to JupyROOT 6.12/06


0L

# Environment Path and Options

In [2]:
NTUPLEDIR   = "/Users/shoh/Projects/CMS/PhD/Analysis/SSL/datav8-skim/"
LUMI        = 35800. # pb-1                                                                                                                
data = ["data_obs"]
#back = [ "ttV", "WJetsToLNu", "TTbar-SL", "WW", "WZ", "ST", "TTbar-DiLep", "DYJetsToLL" ]                                                 
back = [ "ttV" , "WW" ,"WZ" , "TTbar-SL", "ST", "TTbar-DiLep", "WJetsToLNu" , "DYJetsToLL" ]
sign=[]
BLIND=False
POISSON= False
SIGNAL      = 1.
RATIO       = 4
jobs        = []

# Selection option

In [3]:
import pandas as pd
from collections import OrderedDict
from IPython.display import display, HTML
pd.set_option('display.max_colwidth',10000)
ALLTABLE={}
TABLE=[]
#Selection=['OSSF','OSOF','SSSF','SSOF']
for sel in selection:
    TABLE.append(OrderedDict({
        'Selection Region' : sel,
        'Description' : selection[sel]
    }))

ALLTABLE=pd.DataFrame(TABLE)
print "CHOOSE SELECTION"
display(ALLTABLE)

CHOOSE SELECTION


,Selection Region,Description
0,Shoot,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( 1==1 && (Muon_charge[0]!=Muon_charge[0]) && abs(Jet_eta)<2.5 && Jet_pt>30 )) ))
1,SSSF,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( 1==1 && (Muon_charge[0]==Muon_charge[1]) )) ))
2,SSOF,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( 1==1 && (Muon_charge[0]==Electron_charge[0]) )) ))
3,Trigger,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( 1==1 ))
4,OSZmass,"(( (( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( 1==1 && (Muon_charge[0]!=Muon_charge[1]) && nMuon==2 )) )) ) && ( 60 < invariantMass(Muon_pt[0], Muon_eta[0], Muon_phi[0], Muon_mass[0], Muon_pt[1], Muon_eta[1], Muon_phi[1], Muon_mass[1]) && invariantMass(Muon_pt[0], Muon_eta[0], Muon_phi[0], Muon_mass[0], Muon_pt[1], Muon_eta[1], Muon_phi[1], Muon_mass[1]) < 120 ))"
5,OSSF,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( (Muon_charge[0]!=Muon_charge[1]) && Muon_pt[0]>30 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.1 )) ))
6,OSOF,(( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( 1==1 && ( Muon_charge[0]!=Electron_charge[0] ) && Muon_pt[0]>27 && Electron_pt[0]>25 && Muon_pfRelIso03_all[0]<0.1 && Muon_mediumId[0]==1 && Electron_pfRelIso03_all[0]<0.1 )) ))


In [4]:
#var="MHTju_pt"
var="Zmass"
cr="OSSF"
cut=selection[cr]
primaryDataset = getPrimaryDataset(cut)
print "Selected", selection[cr]

Selected (( (HLT_IsoMu22 || HLT_IsoMu22_eta2p1 || HLT_IsoTkMu22_eta2p1 || HLT_IsoMu24 || HLT_IsoTkMu24 || HLT_Mu45_eta2p1 || HLT_Mu50) ) && ( (( nJet>2 ) && ( (Muon_charge[0]!=Muon_charge[1]) && Muon_pt[0]>30 && Muon_pt[1]>15 && Muon_pfRelIso03_all[0]<0.1 )) ))


In [5]:
CANVAS=[]
for var in ['Zmass','Muon_pt[0]']:
    
    if not BLIND:
        processes=data+back
    elif BLIND:
        processes=back

    if not sign:
        Histlist=ProjectDraw(var, cut, LUMI, processes, primaryDataset, NTUPLEDIR)
    else:
        Histlist=ProjectDraw(var, cut, LUMI, processes, primaryDataset, NTUPLEDIR)
    if len(back)>0:
    #If data_obs present, dummy BkgSum == first background process                                                                                                                                    
        Histlist['BkgSum'] = Histlist['data_obs'].Clone("BkgSum") if 'data_obs' in Histlist else Histlist[back[0]].Clone("BkgSum")
        Histlist['BkgSum'].Reset("MICES")
        Histlist['BkgSum'].SetFillStyle(3003)
        Histlist['BkgSum'].SetFillColor(1)
    for i, s in enumerate(back): Histlist['BkgSum'].Add(Histlist[s])

    if len(back)==0 and len(data)==0:
        for i, s in enumerate(sign):
            Histlist[s].Scale(1./Histlist[s].Integral())
            Histlist[s].SetFillStyle(0)
            
    if len(data+back)>0:
        out = draw(Histlist, data if not BLIND else [], back, sign, SIGNAL, RATIO, POISSON, variable[var]['log'])
        out[0].cd(1)
        drawCMS(LUMI, "Preliminary")
        out[0].Update()
        printTable(Histlist, sign)
    else:
        out = drawSignal(Histlist, sign, variable[var]['log'])
        out[0].cd(1)
        drawCMS(LUMI, "Simulation")
        out[0].Update()
    CANVAS.append(out[0])

DATA, NO HLT :  SingleMuonRun2016B-03Feb2017_ver1-v1
DATA, NO HLT :  SingleMuonRun2016B-03Feb2017_ver2-v2
DATA, NO HLT :  SingleMuonRun2016C-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016D-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016E-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016F-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016G-03Feb2017-v1
DATA, NO HLT :  SingleMuonRun2016H-03Feb2017_ver2-v1
DATA, NO HLT :  SingleMuonRun2016H-03Feb2017_ver3-v1
MC, With HLT :  TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8_ext1-v3
MC, With HLT :  TTZToLLNuNu_M-10_TuneCUETP8M1_13TeV-amcatnlo-pythia8_ext1-v1
MC, With HLT :  ttHToNonbb_M125_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8-v1
MC, With HLT :  WW_TuneCUETP8M1_13TeV-pythia8_ext1-v1
MC, With HLT :  WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8-v1
MC, With HLT :  TTJets_SingleLeptFromT_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_ext1-v1
MC, With HLT :  TTJets_SingleLeptFromTbar_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_ext1-v1
MC, With HLT :

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
Warning in <TROOT::Append>: Replacing existing TH1: SingleMuonRun2016B-03Feb2017_ver1-v1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8_ext1-v3 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: WW_TuneCUETP8M1_13TeV-pythia8_ext1-v1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8-v1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: TTJets_SingleLeptFromT_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_ext1-v1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ST_s-channel_4f_leptonDecays_13TeV-amcatnlo-pythia8_TuneCUETP8M1-v1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: TTJets_DiLept_Tu

In [6]:
#ROOT.enableJSVis()
#ROOT>disableJSVis()

In [10]:
CANVAS[0].Draw()

AttributeError: 'PyROOT_NoneType' object has no attribute 'Draw'